In [10]:
import yaml
import re

# Load the SME evaluations from the YAML file
with open('../SME_Evaluations_MentalHealth.yaml', 'r') as file:
    sme_evaluations = yaml.safe_load(file)


In [11]:
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

# from openai import OpenAI
# import json
# from tqdm import tqdm

# client = OpenAI()

# # Load the SME evaluations from the YAML file
# with open('sme-evaluations.yaml', 'r') as file:
#     sme_evaluations = yaml.safe_load(file)

# def generate(instruction, model="gpt-4o"):
#     completion = client.chat.completions.create(
#         model=model,
#         messages=[
#             {"role": "user", "content": 'Does the following text contain any markdown? If so, replace it with simple html tags. Otherwise, leave it as is. Return as JSON with the key "updated" \n\n ```' + instruction + '```'}
#         ],
#         response_format={ "type": "json_object" }
#     )
#     return completion.choices[0].message.content

# for x in tqdm(sme_evaluations['prompts']):
#     response_1 = x['outputs'][0]
#     response_2 = x['outputs'][1]
#     response_1_fixed = generate(response_1)
#     response_2_fixed = generate(response_2)
#     x['outputs'][0] = response_1_fixed
#     x['outputs'][1] = response_2_fixed
#     x['updated_outputs'] = [json.loads(response_1_fixed)['updated'], json.loads(response_2_fixed)['updated']]

# with open('updated-sme-evaluations.yaml', 'w') as file:
#     yaml.dump(sme_evaluations, file)


In [12]:
import yaml

with open('../SME_Evaluations_MentalHealth.yaml', 'r') as file:
    sme_evaluations = yaml.safe_load(file)

# Read template files
survey_template = open("qualtrics_survey_template.txt").read()
instruction_template = open("instruction_template.txt").read()
aspect_question_template = open("aspect_question_template.txt").read()
pre_interview_survey = open("pre_interview_survey.txt").read()
tutorial_page = open("tutorial_page.txt").read()
thank_you_page = open("thankyou_page.txt").read()

questions = []
question_counter = 1

for example in sme_evaluations['prompts']:
    instruction = example['prompt']
    instruction_id = f"Q{question_counter}"
    
    response_1 = example['outputs'][0]['text'].replace('\n', '<br/>')
    response_1_id = example['outputs'][0]['id']
    
    response_2 = example['outputs'][1]['text'].replace('\n', '<br/>')
    response_2_id = example['outputs'][1]['id']
    
    prompt_id = f"Prompt{instruction_id}_a{response_1_id}_b{response_2_id}"

    aspect_questions = example['aspect_questions']
    aspect_questions.insert(0, "Which response is overall better?")

    filled_aspect_questions = []
    question_label = ['a', 'b', 'c']

    for idx, aq in enumerate(aspect_questions):
        mc_question_id = f"Q{question_counter}_{question_label[idx % len(question_label)]}"
        te_question_id = f"T{question_counter}_{question_label[idx % len(question_label)]}"
        filled_aspect_questions.append(aspect_question_template.format(aspect_question=aq, mc_question_id=mc_question_id, te_question_id=te_question_id))

    filled_aspect_questions_str = "\n".join(filled_aspect_questions)

    filled_instruction_str = instruction_template.format(
        question_number=question_counter,
        instruction_id=prompt_id,
        instruction=instruction,
        response_1=response_1,
        response_2=response_2,
        aspect_questions=filled_aspect_questions_str
    )

    questions.append(filled_instruction_str)
    question_counter += 1

questions_str = "\n".join(questions)

# Combine the consent form, pre-interview survey, and main questions
full_survey = survey_template + "\n" + pre_interview_survey + tutorial_page.format(questions=questions_str) + "\n" + thank_you_page
#print('\n'.join(survey_str.split("\n")[:50]))


In [13]:
with open("updated_qualtrics_survey_mental_health.txt", "w") as file:
    file.write(full_survey)
    print("Survey written to updated_qualtrics_survey.txt")


Survey written to updated_qualtrics_survey.txt
